In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
from torch.optim.lr_scheduler import StepLR

In [2]:
orgin_raw_data_dir = 'C:\\Users\\21945\\PycharmProjects\\Hands-on-EEG\\new_implement\\data\\only_walk_3000'
model_save ='C:\\Users\\21945\\PycharmProjects\\Hands-on-EEG\\new_implement\\model'
pic_dir = 'C:\\Users\\21945\\PycharmProjects\\Hands-on-EEG\\new_implement\\pic'

In [3]:
class EEG_Dataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = orgin_raw_data_dir
        self.file_list = os.listdir(self.data_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        file_path = os.path.join(self.data_dir, file_name)
        data = pd.read_csv(file_path, header=None)
        data = data.values
        data = torch.from_numpy(data)
        label_map = { 'walkbase': 0, 'walkl': 1 ,'walkfocus': 2}
        data_label = label_map[file_name.split('_')[0]]
        return data, data_label

In [5]:
dataset = EEG_Dataset(orgin_raw_data_dir)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)

In [6]:
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(Transformer, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # 取平均作为输出
        x = self.fc(x)
        return x

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [8]:
# # 假设输入数据的维度为(batch_size, sequence_length, input_dim)
batch_size = 64
sequence_length = 3000
input_dim = 32
num_classes = 3
d_model = 3000  # Transformer模型中特征的维度
nhead = 4  # 多头自注意力头数
num_layers = 4  # Transformer编码器层数
model = Transformer(d_model, nhead, num_layers).to(device)
print(model)

Transformer(
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=3000, out_features=3000, bias=True)
    )
    (linear1): Linear(in_features=3000, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=3000, bias=True)
    (norm1): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((3000,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=3000, out_features=3000, bias=True)
        )
        (linear1): Linear(in_features=3000, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=Fal

In [9]:
X = torch.rand(1, 32, 3000, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


In [11]:
learning_rate = 0.001
batch_size = 64
num_epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            global train_loss
            train_loss.append(loss)



def test(dataloader, model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.float())
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    global valid_loss
    valid_loss.append(test_loss)
    global accuracy
    accuracy.append(correct)

In [ ]:
# model = model = EEG_LSTM(input_size=31000, hidden_size=128, num_layers=2,num_classes=train_class_number).to(device)
train_loss = []
valid_loss = []
accuracy = []
for t in range(num_epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    model.train(True)
    train(train_loader, model, loss_fn, optimizer)

    model.train(False)
    test(test_loader, model, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 1.051011  [    0/ 1480]
loss: 1.236993  [  400/ 1480]
loss: 1.014266  [  800/ 1480]
loss: 0.822736  [ 1200/ 1480]
Test Error:
 Accuracy: 39.2%, Avg loss: 1.346116 

Epoch 2
-------------------------------
loss: 1.631283  [    0/ 1480]
loss: 0.681360  [  400/ 1480]
loss: 1.129300  [  800/ 1480]


In [ ]:
signal = pd.read_csv("C:\\Users\\21945\\PycharmProjects\\Hands-on-EEG\\new_implement\\data\\only_walk_3000\\walkfocus_zyy_epocflex_2023_window_51.csv",header=None)
model = Transformer()
model.eval()
input = torch.from_numpy(signal.values)
input = input.unsqueeze(0)
with torch.no_grad():
    output = model(input.float())
    # print(output)
    probabilities = torch.nn.functional.softmax(output, dim=1)
    print(probabilities)
#
# _, pred = torch.max(output, dim=1)  # 找到预测分数最大的类别，得到预测类别
# label_map = { 0: 'walkbase', 1: 'walkl' ,2: 'walkfocus'}
# print(label_map[pred.item()])